# Capstone Project - Cafe in Greece
### Applied Data Science Capstone by IBM/Coursera

## Table of Contents
* Introduction
* Data
* Methodology & Analysis
* Results 
* Conclusion

### Introduction 

In the city of Athens, if someone is looking to open a **coffee shop**, a good question is **where** would you recommend that they open it? The background of the problem is that in order for the coffee shop to be **profitable**, there must be enough **customers**, and in order to have enough customers, it is not worth setting it up in the immediate promixity of existing ones.

The audience of this report is explicitly defined to be the coffee shop entrepreuners in Athens as well as big companies that want to open a branch in Greece. They should care about this problem because the location of the new coffee shop has a significant impact on the expected returns on their investment.

### Data

The data used for this project will be extracted form Foursquare, specifically geolocation data. The dataframe will consist from the name of the coffeshops in many different neighborhoods, the shortname, the address, the postal code and lastly the latitude and longitude. The data will be used to develop an unsupervised machine learning model that will determine which areas are optimal for new coffee shops.

### Methodology & Analysis

In this section we will connect to the Foursqare API to extract data about coffee shops in athens, mainly their locations. Next we will depict them on a map.

In [8]:
#Import libraries
import pandas as pd
import numpy as np
import folium
import requests

In [11]:
CLIENT_ID = 'KE0QGFCBPWMPWNGRHDHPWUNBWGISCUHXRRR5WZBKNNG4BVC4'
CLIENT_SECRET = 'REGG2LDRFQV4O4R5AO54F200QGUB5JE14HLGBK52JK5XT3FD'
request_parameters = {
    "client_id": CLIENT_ID,
    "client_secret": CLIENT_SECRET,
    "v": '20180605',
    "section": "coffee",
    "near": "Athens",
    "radius": 1000,
    "limit": 50}

data = requests.get("https://api.foursquare.com/v2/venues/explore", params=request_parameters)

In [12]:
d = data.json()["response"]
d.keys()

dict_keys(['suggestedFilters', 'geocode', 'headerLocation', 'headerFullLocation', 'headerLocationGranularity', 'query', 'totalResults', 'suggestedBounds', 'groups'])

In [13]:
d["headerLocationGranularity"], d["headerLocation"], d["headerFullLocation"]

('city', 'Athens', 'Athens')

In [14]:
d["suggestedBounds"], d["totalResults"]

({'ne': {'lat': 37.98655822362449, 'lng': 23.72819810234214},
  'sw': {'lat': 37.97294398192658, 'lng': 23.705742928296907}},
 119)

In [15]:
d["geocode"]

{'what': '',
 'where': 'athens',
 'center': {'lat': 37.97945, 'lng': 23.71622},
 'displayString': 'Athens, Attica, Greece',
 'cc': 'GR',
 'geometry': {'bounds': {'ne': {'lat': 38.070916906916594,
    'lng': 23.851752904777918},
   'sw': {'lat': 37.79618901343447, 'lng': 23.632542}}},
 'slug': 'athens-greece',
 'longId': '72057594038192307'}

In [16]:
d["groups"][0].keys()

dict_keys(['type', 'name', 'items'])

In [17]:
d["groups"][0]["type"], d["groups"][0]["name"]

('Recommended Places', 'recommended')

In [18]:
items = d["groups"][0]["items"]
print("number of items: %i" % len(items))
items[0]

number of items: 50


{'reasons': {'count': 0,
  'items': [{'summary': 'This spot is popular',
    'type': 'general',
    'reasonName': 'globalInteractionReason'}]},
 'venue': {'id': '58947ad4469aef72f3b2f693',
  'name': 'Holy Moly',
  'location': {'address': 'Ιερά Οδός 21',
   'lat': 37.97953295417367,
   'lng': 23.714712327590686,
   'labeledLatLngs': [{'label': 'display',
     'lat': 37.97953295417367,
     'lng': 23.714712327590686}],
   'cc': 'GR',
   'neighborhood': 'Γκάζι',
   'country': 'Ελλάδα',
   'formattedAddress': ['Ιερά Οδός 21', 'Ελλάδα']},
  'categories': [{'id': '53e0feef498e5aac066fd8a9',
    'name': 'Street Food Gathering',
    'pluralName': 'Street Food Gatherings',
    'shortName': 'Street Food Gathering',
    'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/streetfood_',
     'suffix': '.png'},
    'primary': True}],
  'photos': {'count': 0, 'groups': []}},
 'referralId': 'e-5-58947ad4469aef72f3b2f693-0'}

In [19]:
items[1]

{'reasons': {'count': 0,
  'items': [{'summary': 'This spot is popular',
    'type': 'general',
    'reasonName': 'globalInteractionReason'}]},
 'venue': {'id': '529a0fa811d216d1bea7f4e9',
  'name': 'КОЯØБA (Korova)',
  'location': {'address': 'Κεραμεικού 93-95',
   'crossStreet': 'Πλαταιών',
   'lat': 37.98083850348248,
   'lng': 23.716898746292802,
   'labeledLatLngs': [{'label': 'display',
     'lat': 37.98083850348248,
     'lng': 23.716898746292802}],
   'postalCode': '104 35',
   'cc': 'GR',
   'city': 'Κεραμεικός',
   'state': 'Αττική',
   'country': 'Ελλάδα',
   'formattedAddress': ['Κεραμεικού 93-95 (Πλαταιών)',
    '104 35 Κεραμεικός, Αττική',
    'Ελλάδα']},
  'categories': [{'id': '4bf58dd8d48988d16d941735',
    'name': 'Café',
    'pluralName': 'Cafés',
    'shortName': 'Café',
    'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/cafe_',
     'suffix': '.png'},
    'primary': True}],
  'photos': {'count': 0, 'groups': []},
  'venuePage': {'id': '84693391'}},

We can create the dataframe with a simple for loop

In [24]:
df_raw = []
for item in items:
    venue = item["venue"]
    categories, uid, name, location = venue["categories"], venue["id"], venue["name"], venue["location"]
    assert len(categories) == 1
    shortname = categories[0]["shortName"]
    if not "postalCode" in location:
        continue
    postalcode = location["postalCode"]
    lat = location["lat"]
    lng = location["lng"]
    datarow = (uid, name, shortname, address, postalcode, lat, lng)
    df_raw.append(datarow)
df = pd.DataFrame(df_raw, columns=["uid", "name", "shortname", "address", "postalcode", "lat", "lng"])
print("found %i cafes" % len(df))
df

found 44 cafes


,uid,name,shortname,address,postalcode,lat,lng
0,529a0fa811d216d1bea7f4e9,КОЯØБA (Korova),Café,Μεγάλου Αλεξάνδρου 124,104 35,37.980839,23.716899
1,52b45b33498ec4eceb367aff,Μπλε Παπαγάλος,Bar,Μεγάλου Αλεξάνδρου 124,104 36,37.982779,23.720052
2,5d14fbf2dbde110025847b45,Το Λοκάλι,Café,Μεγάλου Αλεξάνδρου 124,105 53,37.978743,23.721690
3,5d1f0a716438030023f1c006,The Place By Coco-Mat.Bike,Café,Μεγάλου Αλεξάνδρου 124,118 54,37.979279,23.711910
4,4f96b2b11081ac6f2b207c58,O&B Athens All Day Bar Restaurant,Restaurant,Μεγάλου Αλεξάνδρου 124,105 54,37.977500,23.721939
5,597a361e86f4cc5e0c9df477,Saorsa,Cocktail,Μεγάλου Αλεξάνδρου 124,104 36,37.982657,23.720263
6,549c4bb5498e683bd43ea38c,The Underdog,Coffee Shop,Μεγάλου Αλεξάνδρου 124,118 51,37.974889,23.719219
7,4baeb043f964a520f5ce3be3,Gazi College,Café,Μεγάλου Αλεξάνδρου 124,118 54,37.978694,23.710789
8,4e91c29630f81ec6fff7c85e,Δυάρι Café,Café,Μεγάλου Αλεξάνδρου 124,104 35,37.981420,23.713956
9,5a79a94d60d11b0ed798d9c5,Coffee Island,Coffee Shop,Μεγάλου Αλεξάνδρου 124,118 54,37.980569,23.711946


In [22]:
athens_center = d["geocode"]["center"]
athens_center

{'lat': 37.97945, 'lng': 23.71622}

Now we will create a map of Athens with the lat and long values 

In [23]:
from folium import plugins

# create map of Athens using latitude and longitude values
map_athens = folium.Map(location=[athens_center["lat"], athens_center["lng"]], zoom_start=14)

def add_markers(df):
    for (j, row) in df.iterrows():
        label = folium.Popup(row["name"], parse_html=True)
        folium.CircleMarker(
            [row["lat"], row["lng"]],
            radius=5,
            popup=label,
            color='blue',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7,
            parse_html=False).add_to(map_athens)

add_markers(df)
hm_data = df[["lat", "lng"]].as_matrix().tolist()
map_athens.add_child(plugins.HeatMap(hm_data))

map_athens

C:\Users\xarhi\Anaconda3\lib\site-packages\ipykernel_launcher.py:20: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.


### Results

Based on the above map a good location for a new coffee shop would be Omonia, or "Ομόνοια" in greek. We can easily find latitude and longitude for this location from Google Maps.

In [30]:
lat = 37.986133
lng = 23.728267
map_athens = folium.Map(location=[lat, lng], zoom_start=17)
add_markers(df)
folium.CircleMarker(
    [lat, lng],
    radius=15,
    popup="Optimal location for Cafe",
    color='red',
    fill=True,
    fill_color='#3186cc',
    fill_opacity=0.7,
    parse_html=False).add_to(map_athens)
map_athens

### Discussion

In this project, data was extracted from the Foursquare API to create a map with the locations of cafes in Athens, Greece. By visualizing the locations, we have found a more isolated place that is near the center of the city where little to no cafes exist. Hence we propose this location for a new coffee shop! 